# 📈 Описание решения (Solution Description)
В рамках задачи распознавания символов на изображениях **CAPTCHA** были протестированы две архитектуры:

Простая сверточная нейросеть **(SimpleCNN)**

**ResNet18**, адаптированная под маленькие изображения

## 🔹 Базовая модель (SimpleCNN)
Модель состояла из двух сверточных слоёв с ReLU и MaxPooling, за которыми следовали два полносвязных слоя.
Она достигла валидационной точности ~60% и использовалась как начальный ориентир (baseline).
## 🔹 Улучшенная модель (ResNet18)
Для улучшения результатов была использована архитектура ResNet18, адаптированная под входные изображения размером 48×48:

* Первый сверточный слой был изменён (stride=1, kernel_size=3)
* Удалён первый MaxPooling, чтобы сохранить больше пространственной информации
* Модель обучалась с нуля на 20 000 размеченных изображениях (images.npy, labels.npy)

с использованием:
* **CrossEntropyLoss** как функции потерь
* Оптимизатора **Adam**
* **15 эпох обучения**  

## 📈 Результат
ResNet18 достигла валидационной точности 93.5% на отложенной выборке (20% от обучающих данных).

## 📦 Предсказания
После обучения модель была применена к 50 000 тестовым изображениям из файла images_sub.npy.
Предсказания были сохранены в файл submission.csv в формате Kaggle (колонки Id, Category).



In [26]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split



In [27]:
# Загрузка данных
images = np.load('/kaggle/input/dl-captcha-new/mds-misis-dl-captchan/images.npy')
labels = np.load('/kaggle/input/dl-captcha-new/mds-misis-dl-captchan/labels.npy')



In [28]:
if labels.ndim == 2:
    labels = labels.reshape(-1)

# Делим на train/val
images_train, images_val, labels_train, labels_val = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels)

In [29]:
# Датасет

import torch.nn as nn
import torch.nn.functional as F

class CaptchaDataset(Dataset):
    def __init__(self, images, labels=None):
        self.images = images.astype(np.float32) / 255.0
        self.labels = labels
        self.is_test = labels is None

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = torch.tensor(self.images[idx]).permute(2, 0, 1)
        if self.is_test:
            return img
        else:
            label = torch.tensor(self.labels[idx]).long()
            return img, label

In [30]:
# Загрузчики данных

train_dataset = CaptchaDataset(images_train, labels_train)
val_dataset = CaptchaDataset(images_val, labels_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [32]:
# простая CNN Модель 

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 12 * 12, 256)
        self.fc2 = nn.Linear(256, 26)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [34]:
# Обучение модели 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    correct = 0
    total = 0
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = correct / total

    # Валидация
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = val_correct / val_total
    print(f"📚 Epoch {epoch+1} — Loss: {running_loss:.4f} — Train Acc: {train_acc:.4f} — Val Acc: {val_acc:.4f}")

📚 Epoch 1 — Loss: 806.2863 — Train Acc: 0.0538 — Val Acc: 0.0988
📚 Epoch 2 — Loss: 667.4092 — Train Acc: 0.2183 — Val Acc: 0.3202
📚 Epoch 3 — Loss: 487.1010 — Train Acc: 0.4274 — Val Acc: 0.4437
📚 Epoch 4 — Loss: 384.9500 — Train Acc: 0.5425 — Val Acc: 0.5240
📚 Epoch 5 — Loss: 316.2891 — Train Acc: 0.6199 — Val Acc: 0.5535
📚 Epoch 6 — Loss: 268.1502 — Train Acc: 0.6713 — Val Acc: 0.5920
📚 Epoch 7 — Loss: 225.2938 — Train Acc: 0.7269 — Val Acc: 0.5978
📚 Epoch 8 — Loss: 193.6763 — Train Acc: 0.7626 — Val Acc: 0.6042
📚 Epoch 9 — Loss: 159.5098 — Train Acc: 0.8033 — Val Acc: 0.6095
📚 Epoch 10 — Loss: 134.6779 — Train Acc: 0.8345 — Val Acc: 0.6052
📚 Epoch 11 — Loss: 109.5671 — Train Acc: 0.8647 — Val Acc: 0.6070
📚 Epoch 12 — Loss: 90.5222 — Train Acc: 0.8934 — Val Acc: 0.6152
📚 Epoch 13 — Loss: 72.5953 — Train Acc: 0.9141 — Val Acc: 0.6058
📚 Epoch 14 — Loss: 58.4031 — Train Acc: 0.9334 — Val Acc: 0.6115
📚 Epoch 15 — Loss: 47.8825 — Train Acc: 0.9473 — Val Acc: 0.6072


In [35]:
# Пробуем использовать Resnet18

from torchvision.models import resnet18
import torch.nn as nn

# Загружаем модель без предобучения
model = resnet18(pretrained=False)

# Адаптируем под наше изображение 48x48
# Заменим первый сверточный слой на меньший stride
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()  # Убираем maxpool (съедает слишком много на маленьких картинках)

# Выходной слой на 26 классов
model.fc = nn.Linear(512, 26)

# Переносим на устройство
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [36]:
# Оптимизатор и обучение

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [37]:
num_epochs = 15  # Можно увеличить до 20–30 для ResNet

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_correct = 0
    train_total = 0

    # Обучение на train_loader
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_acc = train_correct / train_total

    # Валидация на val_loader
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = val_correct / val_total

    print(f"📚 Epoch [{epoch+1}/{num_epochs}] | Loss: {running_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

📚 Epoch [1/15] | Loss: 827.8809 | Train Acc: 0.0463 | Val Acc: 0.0555
📚 Epoch [2/15] | Loss: 705.7232 | Train Acc: 0.1634 | Val Acc: 0.4420
📚 Epoch [3/15] | Loss: 216.5530 | Train Acc: 0.7382 | Val Acc: 0.8233
📚 Epoch [4/15] | Loss: 91.7439 | Train Acc: 0.8892 | Val Acc: 0.8795
📚 Epoch [5/15] | Loss: 55.9252 | Train Acc: 0.9300 | Val Acc: 0.8592
📚 Epoch [6/15] | Loss: 34.7496 | Train Acc: 0.9575 | Val Acc: 0.8990
📚 Epoch [7/15] | Loss: 27.2388 | Train Acc: 0.9663 | Val Acc: 0.9165
📚 Epoch [8/15] | Loss: 15.3831 | Train Acc: 0.9821 | Val Acc: 0.9005
📚 Epoch [9/15] | Loss: 16.3852 | Train Acc: 0.9790 | Val Acc: 0.9035
📚 Epoch [10/15] | Loss: 9.8214 | Train Acc: 0.9884 | Val Acc: 0.9032
📚 Epoch [11/15] | Loss: 14.6599 | Train Acc: 0.9806 | Val Acc: 0.9005
📚 Epoch [12/15] | Loss: 8.6369 | Train Acc: 0.9903 | Val Acc: 0.9283
📚 Epoch [13/15] | Loss: 7.3196 | Train Acc: 0.9912 | Val Acc: 0.9350
📚 Epoch [14/15] | Loss: 10.0025 | Train Acc: 0.9882 | Val Acc: 0.9265
📚 Epoch [15/15] | Loss: 5.123

In [39]:

# === 3. DataLoader ===
test_dataset = CaptchaDataset(images_sub)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:

# === 4. Предсказания модели ===
model.eval()
all_preds = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cuda().numpy())

In [46]:
images_sub = np.load('/kaggle/input/dl-captcha-new/mds-misis-dl-captchan/images_sub.npy')
print("✅ Размер test-данных:", images_sub.shape)  # Должно быть (50000, 48, 48, 3)

✅ Размер test-данных: (50000, 48, 48, 3)


In [47]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# Класс датасета
class CaptchaDataset(Dataset):
    def __init__(self, images):
        self.images = images.astype(np.float32) / 255.0

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = torch.tensor(self.images[idx])
        if img.shape[0] == 48:  # (H, W, C)
            img = img.permute(2, 0, 1)  # → (C, H, W)
        return img



In [48]:
# DataLoader
test_dataset = CaptchaDataset(images_sub)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)



In [49]:
# Получение предсказаний
model.eval()
all_preds = []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        outputs = model(batch)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())



In [51]:
# Проверка
assert len(all_preds) == 50000, f"❌ Предсказаний только {len(all_preds)}, а должно быть 50000!"

In [44]:
import pandas as pd

# === 5. Формируем submission.csv ===
submission_df = pd.DataFrame({'Category': all_preds})
submission_df.index.name = 'Id'
submission_df.to_csv('submission.csv')

print("✅ Файл submission.csv успешно создан и готов к загрузке на Kaggle!")

✅ Файл submission.csv успешно создан и готов к загрузке на Kaggle!


In [52]:
# Сохранение submission.csv
submission = pd.DataFrame({'Category': all_preds})
submission.index.name = 'Id'
submission.to_csv('submission.csv')

print("✅ submission.csv создан — 50,000 строк готово для Kaggle.")

✅ submission.csv создан — 50,000 строк готово для Kaggle.
